# MODNet - Portrait Image Matting Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/pdf/2011.11961.pdf)          [![GitHub stars](https://img.shields.io/github/stars/ZHKKKe/MODNet?style=social)](https://github.com/ZHKKKe/MODNet)

<p align="justify">This is a <b>Portrait Image Matting Demo</b> of our paper ''<a href="https://arxiv.org/pdf/2011.11961.pdf">Is a Green Screen Really Necessary for Real-Time Portrait Matting?</a>''. We propose a trimap-free MODNet for portrait matting in real time (on a single GPU). If you want to try portrait video matting with MODNet, please refer to our <b>Real-Time Portrait Video Matting Demo</b> based on WebCam [<a href="https://colab.research.google.com/drive/1Pt3KDSc2q7WxFvekCnCLD8P0gBEbxm6J?usp=sharing">online demo</a>] [<a href="https://github.com/ZHKKKe/MODNet/tree/master/demo/video_matting">offline demo</a>].</p>


<p align="justify">In this demo, we provide a pre-trained MODNet for photographic portrait images (<i>e.g.</i>, the image from <a href="https://www.flickr.com/">Flickr</a>). Of course, this model can also handle the portrait images from other data domains (<i>e.g.</i>, the image captured by smartphone). Note that <b><font color='#FF000'>our model may still fail in some cases (<i>e.g.</i>, complex backgrounds and challenging poses/costumes) due to our relatively small labeled training set (only 3k labeled foregrounds)</font></b>.</p>

### **Let's start!**

## 1. Preparation

<p align="justify">In the top menu of this session, select <b>Runtime -> Change runtime type</b>, and set <b>Hardware Accelerator</b> to <b>GPU</b>.</p>

<p align="justify">Clone the repository, and download the pre-trained model:</p>


In [ ]:
import os

# clone the repository
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet

%cd MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt


/home/cvmlserver/Seohyeon/v-coco/MODNet


## 2. Upload Images

<p align="justify">Upload portrait images to be processed (only PNG and JPG format are supported):</p>

In [ ]:
import shutil
# from google.colab import files

# clean and rebuild the image folders
input_folder = 'demo/image_matting/colab/input'
if os.path.exists(input_folder):
  shutil.rmtree(input_folder)
os.makedirs(input_folder)

output_folder = 'demo/image_matting/colab/output'
if os.path.exists(output_folder):
  shutil.rmtree(output_folder)
os.makedirs(output_folder)

# upload images (PNG or JPG)
# image_names = list(files.upload().keys())
# for image_name in image_names:
#   shutil.move(image_name, os.path.join(input_folder, image_name))


In [ ]:
%cd MODNet/

/home/cvmlserver/Seohyeon/v-coco/MODNet


## 3. Inference

<p align="justify">Run the following command for alpha matte prediction:</p>

In [ ]:
!python -m demo.image_matting.colab.inference \
        --input-path demo/image_matting/colab/input \
        --output-path demo/image_matting/colab/output \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt


Process image: COCO_val2014_000000410337.jpg
Process image: COCO_val2014_000000413349.jpg
Process image: COCO_val2014_000000202228.jpg
Process image: COCO_val2014_000000213276.jpg
Process image: COCO_val2014_000000326726.jpg
Process image: COCO_val2014_000000273708.jpg
Process image: COCO_val2014_000000165133.jpg
Process image: COCO_val2014_000000336695.jpg
Process image: COCO_val2014_000000070256.jpg
Process image: COCO_val2014_000000301766.jpg
Process image: COCO_val2014_000000352496.jpg
Process image: COCO_val2014_000000476258.jpg
Process image: COCO_val2014_000000487338.jpg
Process image: COCO_val2014_000000015725.jpg
Process image: COCO_val2014_000000067180.jpg
Process image: COCO_val2014_000000319908.jpg
Process image: COCO_val2014_000000060548.jpg
Process image: COCO_val2014_000000261050.jpg
Process image: COCO_val2014_000000290113.jpg
Process image: COCO_val2014_000000515038.jpg
Process image: COCO_val2014_000000286544.jpg
Process image: COCO_val2014_000000142924.jpg
Process im

## 4. Visualization

<p align="justify">Display the results (from left to right: image, foreground, and alpha matte):</p>

In [ ]:
import numpy as np
from PIL import Image
import cv2

def combined_display(image, matte, image_name):
  # calculate display resolution
  w, h, c = image.shape
  # w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))
  
  # obtain predicted foreground
  # image = np.asarray(image)
  # if len(image.shape) == 2:
  #   image = image[:, :, None]
  # if image.shape[2] == 1:
  #   image = np.repeat(image, 3, axis=2)
  # elif image.shape[2] == 4:
  #   image = image[:, :, 0:3]
  # matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  # foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  foreground = cv2.bitwise_and(image, matte)
  cv2.imwrite('/home/cvmlserver/Seohyeon/v-coco/MODNet/demo/image_matting/colab/final/'+matte_name, foreground)
  # combine image, foreground, and alpha into one line
  combined = np.concatenate((image, foreground, matte * 255), axis=1)
  combined = Image.fromarray(np.uint8(combined)).resize((rw, rh))
  return foreground

input_folder = '/home/cvmlserver/Seohyeon/v-coco/MODNet/demo/image_matting/colab/input/'
output_folder = '/home/cvmlserver/Seohyeon/v-coco/MODNet/demo/image_matting/colab/output/'
# visualize all images
image_names = os.listdir(input_folder)
for image_name in image_names:
  matte_name = image_name.split('.')[0] + '.png'
  image = cv2.imread(os.path.join(input_folder, image_name))
  matte = cv2.imread(os.path.join(output_folder, matte_name))
  combined_display(image, matte, matte_name)
  # display(combined_display(image, matte, matte_name))
  print(image_name, '\n')


COCO_val2014_000000410337.jpg 

COCO_val2014_000000413349.jpg 

COCO_val2014_000000202228.jpg 

COCO_val2014_000000213276.jpg 

COCO_val2014_000000326726.jpg 

COCO_val2014_000000273708.jpg 

COCO_val2014_000000165133.jpg 

COCO_val2014_000000336695.jpg 

COCO_val2014_000000070256.jpg 

COCO_val2014_000000301766.jpg 

COCO_val2014_000000352496.jpg 

COCO_val2014_000000476258.jpg 

COCO_val2014_000000487338.jpg 

COCO_val2014_000000015725.jpg 

COCO_val2014_000000067180.jpg 

COCO_val2014_000000319908.jpg 

COCO_val2014_000000060548.jpg 

COCO_val2014_000000261050.jpg 

COCO_val2014_000000290113.jpg 

COCO_val2014_000000515038.jpg 

COCO_val2014_000000286544.jpg 

COCO_val2014_000000142924.jpg 

COCO_val2014_000000577073.jpg 

COCO_val2014_000000010142.jpg 

COCO_val2014_000000097748.jpg 

COCO_val2014_000000374369.jpg 

COCO_val2014_000000143541.jpg 

COCO_val2014_000000437609.jpg 

COCO_val2014_000000362506.jpg 

COCO_val2014_000000458992.jpg 

COCO_val2014_000000140743.jpg 

COCO_val

In [ ]:

import shutil
bg_folder = '/home/cvmlserver/Seohyeon/v-coco/Images/'
bg_new = '/home/cvmlserver/Seohyeon/v-coco/backgrouds/'
bg_names = os.listdir(bg_folder)
for bg_name in bg_names:
    if bg_name == '.DS_Store':
            pass
    image_names = os.listdir(os.path.join(bg_folder,bg_name))
    i = 0
    for image_name in image_names[:15]:
        if image_name == '.DS_Store':
            pass
        name = bg_name+'_'+str(i)+'.jpg'
        shutil.move(os.path.join(bg_folder,bg_name, image_name), os.path.join(bg_new, name))
        i+=1

NotADirectoryError: [Errno 20] Not a directory: '/home/cvmlserver/Seohyeon/v-coco/Images/.DS_Store'

In [ ]:
import cv2
import numpy as np


input_folder = '/home/cvmlserver/Seohyeon/v-coco/MODNet/demo/image_matting/colab/final/'
mat_foler = '/home/cvmlserver/Seohyeon/v-coco/MODNet/demo/image_matting/colab/output/'
bg_folder = '/home/cvmlserver/Seohyeon/v-coco/Images/'
# visualize all images
image_names = os.listdir(input_folder)
for image_name in image_names:
    bg = cv2.imread('/home/cvmlserver/Seohyeon/v-coco/GFM/samples/_peluqueria01_w_gra.jpg')
    matte = cv2.imread('/home/cvmlserver/Seohyeon/v-coco/MODNet/demo/image_matting/colab/output/src.png')   
    image = cv2.imread('/home/cvmlserver/Seohyeon/v-coco/MODNet/foreground.png')

    image = cv2.resize( image, (bg.shape[1], bg.shape[0]))
    matte = cv2.resize( matte, (bg.shape[1], bg.shape[0]))
    # roi = bg[50:bg.shape[0]-50, 50:bg.shape[1]-50]
    print(image.shape)
    w, h, channels = image.shape 

    rw, rh = 800, int(h * 800 / (3 * w))

    # obtain predicted foreground
    image = np.asarray(image)
    bg = np.asarray(bg)
    matte = np.asarray(matte)

    matte = cv2.bitwise_not(matte)
    bg = cv2.bitwise_and(bg, matte)
    # image = cv2.bitwise_and(image, matte)
    cv2.imwrite('output.png', image+bg)

(338, 450, 3)


True

## Download Results

<p align="justify">Download the Zip package of predicted alpha mattes:</p>

In [ ]:
zip_filename = 'matte.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)

os.system(f"zip -r -j {zip_filename} {output_folder}/*")
files.download(zip_filename)